## After training the model, we can use this notebook to test the model. 

### 1. Directly download the model and test. Run this notebook in A100 GPU machine (NC24adsA100 compute instance)

In [36]:
import pandas as pd
import json
from torch.utils.data import DataLoader

train_ds = pd.read_json("data/train.jsonl", lines=True).to_dict(orient="records")
def collate(batch):
    batch = [item["input"] for item in batch]


    return batch
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, collate_fn=collate)

In [37]:
for batch in train_loader:
    print(batch)
    break

['<START_Q>A large gathering occurred at the town hall with 200 people participating. 100 people decided to have a snack, and then 20 new outsiders joined in to have a snack. Half of these snack eaters got full and left. 10 new outsiders came to have snacks, too. 30 more snack eaters got their fill and left. Then half of the remaining snack eaters left later on. How many snack eaters are left?<END_Q><START_A>At the meeting, 200-100=<<200-100=100>>100 people decided to have a snack.\n20 new people came in to snack and the area had 100+20=<<100+20=120>>120 of them eating.\n120/2=<<120/2=60>>60 people got full and left.\n10 more new people came to join in the snacking so 60+10=<<10+60=70>>70 people were eating.\nAfter 30 people left, 70-30=<<70-30=40>>40 people who were eating remained.\nIn the end, only 40/2=<<40/2=20>>20 kept eating.\n#### 20<END_A>', '<START_Q>If a tarantula has eight legs, and one tarantula egg sac can contain 1000 tarantulas, how many baby tarantula legs would be in 

In [ ]:
# import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
import time
from azure.ai.ml import MLClient, Input
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component
credential = DefaultAzureCredential()
subscription_id = "840b5c5c-3f4a-459a-94fc-6bad2a969f9d" # your subscription id
resource_group = "ml"#your resource group
workspace = "ws01ent" #your workspace name
workspace_ml_client = MLClient(credential, subscription_id, resource_group, workspace)

In [ ]:
model_name = "llama2_13b_fine_tuned"
model_path="./"
workspace_ml_client.models.download(model_name, version="2",download_path=model_path)
#after this step, remove the redundant parent folder name "llama2_13b_fine_tuned" so that the downloaded folder only has one 

In [ ]:
import mlflow
import pandas as pd
model = mlflow.pyfunc.load_model(model_name)
prompt = """
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate a chessboard with the given size and with pieces at the specified positions.
### Input:
Size: 8x8 \nPiece positions:\nWhite Rook at H8\nWhite Pawn at B3\nBlack Pawn at A3\nBlack Rook at H2

"""
model_input= {"text":[prompt], "max_length":100}
model.predict(model_input)


### 2.Deploy to managed online endpoint and test

1. Create online endpoint: ```az ml online-endpoint create -f deployment/endpoint.yml```
2. Create the deployment: ```az ml online-deployment update -f deployment/deployment.yml```

In [2]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
prompt = """
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Summarize the following input to less than 30 words .
### Input:
In general, perplexity is a measurement of how well a probability model predicts a sample. In the context of Natural Language Processing, perplexity is one way to evaluate language models.
A language model is a probability distribution over sentences: it’s both able to generate plausible human-written sentences (if it’s a good language model) and to evaluate the goodness of already written sentences. Presented with a well-written document, a good language model should be able to give it a higher probability than a badly written document, i.e. it should not be “perplexed” when presented with a well-written document.
Thus, the perplexity metric in NLP is a way to capture the degree of ‘uncertainty’ a model has in predicting (i.e. assigning probabilities to) text."""
data= {"data":{"text":[prompt], "max_length":100}}

body = str.encode(json.dumps(data))

url = 'https://llma2-fine-tuning.westus2.inference.ml.azure.com/score'
# Replace this with the primary/secondary key or AMLToken for the endpoint
api_key = ''
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")

# The azureml-model-deployment header will force the request to go to a specific deployment.
# Remove this header to have the request observe the endpoint traffic rules
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'blue' }

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

b'[[{"generated_text": "\\nBelow is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\\n\\n### Instruction:\\nSummarize the following input to less than 30 words .\\n### Input:\\nIn general, perplexity is a measurement of how well a probability model predicts a sample. In the context of Natural Language Processing, perplexity is one way to evaluate language models.\\nA language model is a probability distribution over sentences: it\\u2019s both able to generate plausible human-written sentences (if it\\u2019s a good language model) and to evaluate the goodness of already written sentences. Presented with a well-written document, a good language model should be able to give it a higher probability than a badly written document, i.e. it should not be \\u201cperplexed\\u201d when presented with a well-written document.\\nThus, the perplexity metric in NLP is a way to capture the degree of \\

: 

Perplexity is a measure of how well a probability model predicts a sample. It is used to evaluate language models in NLP. A good language model should assign higher probabilities to well-written documents than to badly written ones. Perplexity is a way to capture the degree of uncertainty a model has in predicting text

In [7]:
print(data[1])

{'instruction': 'What are the three primary colors?', 'input': '', 'output': 'The three primary colors are red, blue, and yellow.'}
